In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import fcsparser
import math
import time
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from collections import Counter
from scipy.spatial import cKDTree as KDTree
import os
import numba 
from sklearn.metrics import accuracy_score
import random
import numpy as np
import pandas as pd 
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import classification_report
from scipy.cluster.hierarchy import dendrogram, fcluster
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import OPTICS
from glob import glob 
from tqdm import tqdm
from scipy.spatial.distance import pdist , squareform
np.random.seed(5)
import itertools
#from plotnine import *
import sys

In [13]:
number_of_patient = 50
number_of_patient_phenotype = 4
cluster_std_1 = 0.1
number_of_cell = 10000

vectors_of_probability = [[0.7,0.2,0.2],[0.2,0.7,0.9],[0.7,0.9,0.7],[0.2,0.9,0.2]]


for i,x in enumerate(vectors_of_probability):
    globals()[f"vector_of_probability_{i}"] = x

splitted_vector = np.round(np.linspace(0, number_of_patient, number_of_patient_phenotype+1)[1:]).astype(int)
list_of_array_patients = np.split(range(number_of_patient), splitted_vector)[:-1]

patient_and_phenotype = {'Patient_Number':[] ,'Phenotype_Number':[]}

for group_patient_phenotype , list_patient in tqdm(enumerate(list_of_array_patients)):
    vector_of_probability = globals()[f"vector_of_probability_{group_patient_phenotype}"]
    number_of_dimension = len(vector_of_probability)
    for patient in tqdm(list_patient):
        phenotype_continuous_data = []
        phenotype_binary_data = []
        number_of_feature_to_generate = 1

        #Patient_phenotype
        patient_and_phenotype['Patient_Number'].append(f'patient_N°{str(patient).zfill(3)}')
        patient_and_phenotype['Phenotype_Number'].append(group_patient_phenotype+1)


        #First define random state and dimentionality
        for number in range(number_of_dimension):
            globals()[f"random_state{number}"] =  random.randint(0, 100)

        for i in range(len(vector_of_probability)):
            globals()[f"cell_distribution_col_{i}"] = [vector_of_probability[i], 1- vector_of_probability[i]]

        for i in range(number_of_dimension):
            #generate distribution
            distribution = 'cell_distribution_col_'+f'{i}' #integer for the distribution
            cells_distributed = (np.array(globals()[distribution]) * number_of_cell).astype(int) #distribution

            X, y = make_blobs(n_samples=cells_distributed, centers=None, n_features= number_of_feature_to_generate, cluster_std=cluster_std_1, random_state= globals()[f"random_state{i}"])
            phenotype_continuous_data.append(X)
            phenotype_binary_data.append(y)
            #globals()[f"random_state{i}"] += 10

        phenotype_continuous_data_patient = pd.DataFrame(list(map(np.ravel,phenotype_continuous_data))).T
        phenotype_binary_data_patient = pd.DataFrame(list(map(np.ravel,phenotype_binary_data))).T

print(patient_and_phenotype)

pd.DataFrame(patient_and_phenotype)
         

100%|██████████| 12/12 [00:08<00:00,  1.43it/s]
4it [00:34,  8.73s/it]

{'Patient_Number': ['patient_N°000', 'patient_N°001', 'patient_N°002', 'patient_N°003', 'patient_N°004', 'patient_N°005', 'patient_N°006', 'patient_N°007', 'patient_N°008', 'patient_N°009', 'patient_N°010', 'patient_N°011', 'patient_N°012', 'patient_N°013', 'patient_N°014', 'patient_N°015', 'patient_N°016', 'patient_N°017', 'patient_N°018', 'patient_N°019', 'patient_N°020', 'patient_N°021', 'patient_N°022', 'patient_N°023', 'patient_N°024', 'patient_N°025', 'patient_N°026', 'patient_N°027', 'patient_N°028', 'patient_N°029', 'patient_N°030', 'patient_N°031', 'patient_N°032', 'patient_N°033', 'patient_N°034', 'patient_N°035', 'patient_N°036', 'patient_N°037', 'patient_N°038', 'patient_N°039', 'patient_N°040', 'patient_N°041', 'patient_N°042', 'patient_N°043', 'patient_N°044', 'patient_N°045', 'patient_N°046', 'patient_N°047', 'patient_N°048', 'patient_N°049'], 'Phenotype_Number': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

,Patient_Number,Phenotype_Number
0,patient_N°000,1
1,patient_N°001,1
2,patient_N°002,1
3,patient_N°003,1
4,patient_N°004,1
5,patient_N°005,1
6,patient_N°006,1
7,patient_N°007,1
8,patient_N°008,1
9,patient_N°009,1


In [12]:
cells_distributed

array([2000, 8000])

In [23]:

path_to_store_frame = 'D:/Dossier_These_code_and_datas/LDA_all/Generate_patient_with_our_algorithm/Patient_generated_binary_only_mutliple_gaussian'
number_of_cluster = 4

dirs = path_to_store_frame
a = 'Generated_file'
files = glob(f'{dirs}/{a}*.csv')
list_name = []
list_of_frame_to_append = []
phenotype_code = []
for file in tqdm(files):
    name = '_'.join(file.split("_")[-2:]).replace('.csv','')
    list_name.append(name)

    df_to_generate = pd.read_csv(file, index_col= 0)
    print(df_to_generate
    list_of_frame_to_append.append(df_to_generate)
overall_dataframe = pd.concat(list_of_frame_to_append, ignore_index=False)
overall_dataframe = pd.DataFrame(stats.zscore(overall_dataframe, nan_policy='omit'))


kmeans = KMeans(n_clusters=number_of_cluster, random_state=41).fit(overall_dataframe)
labels = kmeans.labels_
overall_dataframe['K-means labels'] = labels 

list_of_dataframe_sliced = []

start_i = 0
for i in range(number_of_cell,len(overall_dataframe)+number_of_cell,number_of_cell):
    sliced_frame = overall_dataframe.iloc[start_i:i,:]
    list_of_dataframe_sliced.append(sliced_frame)
    start_i =+ i

list_of_series = []

for i,frame in enumerate(list_of_dataframe_sliced):
    counter = Counter(frame['K-means labels'])
    list_of_series.append(pd.Series(counter))

dataframe_for_LDA = pd.DataFrame(list_of_series, index = list_name )
dataframe_for_LDA.fillna(0, inplace= True)

#return overall_dataframe, dataframe_for_LDA,phenotype_code


100%|██████████| 50/50 [00:00<00:00, 121.18it/s]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [34]:
for file in tqdm(files):
    name = '_'.join(file.split("_")[-2:]).replace('.csv','')
    list_name.append(name)

    df_to_generate = pd.read_csv(file, index_col= 0)
    print(df_to_generate.isna().sum() )
    list_of_frame_to_append.append(df_to_generate)

 22%|██▏       | 11/50 [00:00<00:00, 106.77it/s]

0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64

 66%|██████▌   | 33/50 [00:00<00:00, 105.51it/s]


0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    0
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64

100%|██████████| 50/50 [00:00<00:00, 104.14it/s]


0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64
0    1
1    0
2    0
dtype: int64


In [30]:
overall_dataframe = pd.concat(list_of_frame_to_append, ignore_index=False)
overall_dataframe
overall_dataframe = pd.DataFrame(stats.zscore(overall_dataframe, nan_policy='omit'))
overall_dataframe


,0,1,2
0,-0.483906,-1.070694,-1.082014
1,-0.514126,-1.055694,-0.963617
2,-0.505887,-1.026945,-1.121204
3,-0.485679,-0.596502,-1.083586
4,-0.494949,-1.051979,-1.059020
...,...,...,...
1499995,1.239390,1.758814,-1.672458
1499996,-1.138179,-1.470074,-1.683250
1499997,-1.076354,1.788252,-1.681926
1499998,1.241517,-1.467994,-1.688233


In [33]:
overall_dataframe.isna().sum() 

0    75
1     0
2     0
dtype: int64

In [45]:
            for i in range(len(vector_of_probability)):
                globals()[f"cell_distribution_col_{i}"] = np.round([vector_of_probability[i], 1- vector_of_probability[i]],3)
                print(globals()[f"cell_distribution_col_{i}"])

[0.2 0.8]
[0.9 0.1]
[0.2 0.8]


In [46]:
for i in range(number_of_dimension):
    #generate distribution
    distribution = 'cell_distribution_col_'+f'{i}' #integer for the distribution
    cells_distributed = (np.array(globals()[distribution]) * number_of_cell).astype(int) #distribution
    print(distribution)


cell_distribution_col_0
cell_distribution_col_1
cell_distribution_col_2


In [54]:
cell = 0.1

print(cell)

cell += 1
print(cell)

cell += 1

print(cell)

0.1
1.1
2.1


In [48]:
cell

1